<a href="https://colab.research.google.com/github/lkozina1309/tensorflow/blob/main/CNN/Jupyter/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script *image_classification.ipynb* is used to classify horses or humans on an images. It uses tensorflow dataset *horses_or_humans*. 

We'll import libraries first.

In [23]:
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

Then, we have download the dataset.

In [24]:
tfds.disable_progress_bar()

(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'horses_or_humans',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [25]:
IMG_SIZE = 160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

In [26]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [27]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

Now, we have to download pretrained model.

In [28]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')



And we have to set base model not to be trainable.

In [29]:
base_model.trainable = False

Now, we are creating the model and compiling it.

In [30]:
model = tf.keras.Sequential([
  base_model,
  tf.keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1)
])

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

Evaluate loss and accuracy.

In [33]:
loss, acc = model.evaluate(validation_batches, steps = 20)

 4/20 [=====>........................] - ETA: 5s - loss: 0.5160 - accuracy: 0.8137WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 20 batches). You may need to use the repeat() function when building your dataset.


20/20 [==============================] - 4s 56ms/step - loss: 0.5160 - accuracy: 0.8137


And print the accuracy at the end.

In [34]:
history = model.fit(train_batches,
                    epochs=3,
                    validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

Epoch 1/3
26/26 [==============================] - 21s 567ms/step - loss: 0.4076 - accuracy: 0.8504 - val_loss: 0.3372 - val_accuracy: 0.9216
Epoch 2/3
26/26 [==============================] - 18s 567ms/step - loss: 0.2965 - accuracy: 0.9258 - val_loss: 0.2582 - val_accuracy: 0.9706
Epoch 3/3
26/26 [==============================] - 18s 567ms/step - loss: 0.2228 - accuracy: 0.9684 - val_loss: 0.1942 - val_accuracy: 0.9804
[0.8503649830818176, 0.9257907271385193, 0.9683698415756226]
